In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, c8bdabc2-5047-4896-b80e-ca66c965114e)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 4, Finished, Available)

In [3]:
model = (AnalyzeText()
        .setTextCol('Description')
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 5, Finished, Available)

In [4]:
result=model.transform(df)

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 6, Finished, Available)

In [5]:
display(result)

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0862a239-097a-4b9c-846b-032690f6c5e7)

In [6]:
from pyspark.sql.functions import col

sentiment= result.withColumn("Sentiment",col("response.documents.sentiment"))
display(sentiment)

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, b36bb11f-2d99-4e3e-904c-6d5b72f4eb74)

In [7]:
sentiment_final=sentiment.drop('error','response')
display(sentiment_final)

StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3e35521c-dfb7-4177-950e-3497f9192e05)

In [8]:
from pyspark.sql.utils import AnalysisException

try:
    table='Bing_lake_db.tbl_sentiment_analysis'
    sentiment_final.write.format('delta').saveAsTable(table)

except AnalysisException:
    print('Table Already exist')

    sentiment_final.createOrReplaceTempView('vw_sentiment_final')

    spark.sql(f"""  MERGE INTO {table} target_table USING vw_sentiment_final source_view ON source_view.url = target_table.url 
                    WHEN MATCHED AND 
                    source_view.Title <> target_table.Title OR 
                    source_view.Description <> target_table.Description OR
                    source_view.Category <> target_table.Category OR
                    source_view.Url <> target_table.URL OR
                    source_view.Provider <> target_table.Provider OR
                    source_view.DatePublished <> target_table.DatePublished
                    Then UPDATE SET *

                    when not matched then insert *
                    """)



StatementMeta(, afec1df1-77c0-4c86-9021-42a8d2f9e0f0, 10, Finished, Available)